Authors: *Alessia Cotroneo - Alessandro Del Vitto- Michele Di Sabato*

In this notebook we implement a pure (no fine tuning) Transfer Learning, using InceptionV3 architecture.


In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
%cd /gdrive/MyDrive/path/to/your/folder

/gdrive/MyDrive/Colab Notebooks/ANN&DL First Challenge


In [ ]:
!unzip dataset.zip

In [ ]:
dataset_dir = 'training'

## Import libraries and set seeds for reproducibility

In [ ]:
# import the relevant libraries
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

# Random seed for reproducibility
seed = 123

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

2.7.0


In [ ]:
def get_next_batch(generator):
    # get new batch of data and plot it
  batch = next(generator)

  image = batch[0]
  target = batch[1]

  print("(Input) image shape:", image.shape)
  print("Target shape:",target.shape)

  # Visualize only the first sample
  image = image[0]
  target = target[0]
  target_idx = np.argmax(target)
  print()
  print("Categorical label:", target)
  print("Label:", target_idx)
  print("Class name:", labels[target_idx])
  
  # plot the color intensity of the "central" pixel to check if preprocessing worked
  print("Middle Pixel Value R: ", image[125][125][0])
  print("Middle Pixel Value G: ", image[125][125][1])
  print("Middle Pixel Value B: ", image[125][125][2])
  fig = plt.figure(figsize=(6, 4))
  plt.imshow(np.uint8(image))

  return batch

## Model's hyperparameters

In [ ]:
target_size = 256
input_shape = (target_size, target_size, 3)
epochs = 50
batch_size = 16

## Function that creates checkpoints and implements EarlyStopping

In [ ]:
# Utility function to create folders and callbacks for training
from datetime import datetime

def create_folders_and_callbacks(model_name,transfer_learning=False):
    
  # creates folders and callbacks, collected into a list, since the argument "callbacks" in the function .fit() requires a list
  
  #  if transfer_learning = True this function creates a folder called "CNN_TL", otherwise the name is just "CNN"

  if transfer_learning:
    exps_dir = os.path.join('CNN_TL')
  else:
    exps_dir = os.path.join('CNN')
  if not os.path.exists(exps_dir):
      os.makedirs(exps_dir) # creates and SETS the directory if not found

  now = datetime.now().strftime('%b%d_%H-%M-%S')

  exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
  if not os.path.exists(exp_dir):
      os.makedirs(exp_dir)
      
  callbacks = []

  # Model checkpoint
  # ----------------
  ckpt_dir = os.path.join(exp_dir, 'ckpts')
  if not os.path.exists(ckpt_dir):
      os.makedirs(ckpt_dir)

  ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'), 
                                                     save_weights_only=False, # True to save only weights
                                                     save_best_only=False) # True to save only the best epoch 
  callbacks.append(ckpt_callback)

  # Visualize Learning on Tensorboard
  # ---------------------------------
  tb_dir = os.path.join(exp_dir, 'tb_logs')
  if not os.path.exists(tb_dir):
      os.makedirs(tb_dir)
      
  # By default shows losses and metrics for both training and validation
  tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir, 
                                               profile_batch=0,
                                               histogram_freq=1)  # if > 0 (epochs) shows weights histograms
                                                                  # histogram_freq = 5 means that i want to save 1 in 5 epochs (e.g. 1, 6, 6+5, ...)
                                                                  # histogram_freq = 1 means that i want to save 1 in 5 epochs (e.g. 1, 2, 3, ...)
  callbacks.append(tb_callback)

  # Early Stopping
  # --------------
  es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode = "max", patience=10, restore_best_weights=True)
  #es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode = "min", patience=30, restore_best_weights=True)
  callbacks.append(es_callback)

  return callbacks

## Training and Validation splitting in the context of Transfer Learning

In [ ]:
labels = ['Apple','Blueberry','Cherry','Corn','Grape','Orange','Peach','Pepper','Potato','Raspberry','Soybean','Squash','Strawberry','Tomato']
# parameters for the ImageDataGenerator function (preprocessing)
reshape_mean = False
reshape_sd   = False
use_data_augmentation = True

from tensorflow.keras.applications.inception_v3 import preprocess_input

if use_data_augmentation:
  train_data_gen = ImageDataGenerator(validation_split=0.2, samplewise_center = reshape_mean, samplewise_std_normalization = reshape_sd,
                                      rotation_range = 30,
                                      height_shift_range = 50,
                                      width_shift_range = 50,
                                      zoom_range = 0.3,
                                      horizontal_flip = True,
                                      vertical_flip = True,
                                      fill_mode = "constant",
                                      rescale = 1/255.,
                                      preprocessing_function = preprocess_input)
else:
  train_data_gen = ImageDataGenerator(validation_split=0.2,
                                      rescale = 1/255.,
                                      preprocessing_function = preprocess_input)  
target_size = 256
train_gen = train_data_gen.flow_from_directory(directory=dataset_dir,
                                               target_size=(target_size,target_size),
                                               color_mode='rgb',
                                               classes=None,
                                               class_mode='categorical',
                                               batch_size=batch_size,
                                               shuffle=True,
                                               seed=seed,
                                               subset="training")
val_gen = train_data_gen.flow_from_directory(directory=dataset_dir,
                                               target_size=(target_size,target_size),
                                               color_mode='rgb',
                                               classes=None,
                                               class_mode='categorical',
                                               batch_size=batch_size,
                                               shuffle=False,
                                               seed=seed,
                                               subset="validation")

Found 14189 images belonging to 14 classes.
Found 3539 images belonging to 14 classes.


In [ ]:
if False: # used to check is ImageDataGenerator was successful
  _ = get_next_batch(train_gen)

### load InceptionV3

In [ ]:
supernet = tfk.applications.InceptionV3(
    include_top=False,
    weights="imagenet",
    input_shape=(299,299,3) # the input image of InceptionV3 is a 299x299 picture
)

### design the architecture

In [ ]:
supernet.trainable = False # no fine tuning
new_size = 299
input_size = 256

inputs = tfk.Input(shape=(input_size,input_size,3))

x = tfkl.Resizing(new_size, new_size, interpolation="bicubic")(inputs)

x = supernet(x)

x = tfkl.Flatten(name='Flattening')(x)

x = tfkl.Dense(
        128, 
        activation='relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        kernel_regularizer=tf.keras.regularizers.l2(1e-3))(x)

x = tfkl.Dropout(0.2, seed=seed)(x)

x = tfkl.Dense(
        128, 
        activation='relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        kernel_regularizer=tf.keras.regularizers.l2(1e-3))(x)

x = tfkl.Dropout(0.2, seed=seed)(x)

outputs = tfkl.Dense(
        14, 
        activation='softmax',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        kernel_regularizer=tf.keras.regularizers.l2(1e-3))(x)

tf_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# learning rate = 1e-4 since we are using Transfer Learning
tf_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-4), metrics='accuracy')

### summarize the model

In [ ]:
tf_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 resizing (Resizing)         (None, 299, 299, 3)       0         
                                                                 
 inception_v3 (Functional)   (None, 8, 8, 2048)        21802784  
                                                                 
 Flattening (Flatten)        (None, 131072)            0         
                                                                 
 dense (Dense)               (None, 128)               16777344  
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512 

#### **DISCLAIMER**: We have written the following code in a way that it can be generalized to Fine Tuning, even though we don't do it in this Notebook.


In [ ]:
ft_model = tf_model
ft_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 resizing (Resizing)         (None, 299, 299, 3)       0         
                                                                 
 inception_v3 (Functional)   (None, 8, 8, 2048)        21802784  
                                                                 
 Flattening (Flatten)        (None, 131072)            0         
                                                                 
 dense (Dense)               (None, 128)               16777344  
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512 

## choose the layers for the fine tuning

In [ ]:
ft_model.get_layer('inception_v3').trainable = True
# InceptionV3 has 311 layers: if we want to fine tune it, we can change the following parameter to tune a specific number of layers
freeze_up_to_this_index = 311
for i, layer in enumerate(ft_model.get_layer('inception_v3').layers[:freeze_up_to_this_index]):
  layer.trainable=False
# print to check everythin is fine
for i, layer in enumerate(ft_model.get_layer('inception_v3').layers):
   print(i, layer.name, layer.trainable)

0 input_1 False
1 conv2d False
2 batch_normalization False
3 activation False
4 conv2d_1 False
5 batch_normalization_1 False
6 activation_1 False
7 conv2d_2 False
8 batch_normalization_2 False
9 activation_2 False
10 max_pooling2d False
11 conv2d_3 False
12 batch_normalization_3 False
13 activation_3 False
14 conv2d_4 False
15 batch_normalization_4 False
16 activation_4 False
17 max_pooling2d_1 False
18 conv2d_8 False
19 batch_normalization_8 False
20 activation_8 False
21 conv2d_6 False
22 conv2d_9 False
23 batch_normalization_6 False
24 batch_normalization_9 False
25 activation_6 False
26 activation_9 False
27 average_pooling2d False
28 conv2d_5 False
29 conv2d_7 False
30 conv2d_10 False
31 conv2d_11 False
32 batch_normalization_5 False
33 batch_normalization_7 False
34 batch_normalization_10 False
35 batch_normalization_11 False
36 activation_5 False
37 activation_7 False
38 activation_10 False
39 activation_11 False
40 mixed0 False
41 conv2d_15 False
42 batch_normalization_15 False

In [ ]:
ft_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 resizing (Resizing)         (None, 299, 299, 3)       0         
                                                                 
 inception_v3 (Functional)   (None, 8, 8, 2048)        21802784  
                                                                 
 Flattening (Flatten)        (None, 131072)            0         
                                                                 
 dense (Dense)               (None, 128)               16777344  
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512 

In [ ]:
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-4), metrics='accuracy')

## fit the model

In [ ]:
FT_InceptionV3_callbacks = create_folders_and_callbacks(model_name='model_FT', transfer_learning = True)
ft_history = ft_model.fit(
    x = train_gen,
    batch_size = 16,
    epochs = 50,
    validation_data = val_gen,
    callbacks = FT_InceptionV3_callbacks
).history

Epoch 1/50
  1/887 [..............................] - ETA: 2:06:17 - loss: 3.1764 - accuracy: 0.1250

KeyboardInterrupt: ignored